# Computing Bayes factors

This tutorial explains how to use the `bayes_factor` function provided with the package in order to assess the quality of the p-mode fits performed with `apollinaire`. 

> **Note:** You must have run the `first_steps.ipynb` ([Quickstart tutorial](https://apollinaire.readthedocs.io/en/latest/usage/quickstart/first_steps.html)) notebook before running the code in this tutorial (in order to create the hdf5 files that will be used to compute the Bayes factors). 

In [1]:
import apollinaire as apn
import apollinaire.peakbagging as apn_pkb
import numpy as np
import pandas as pd 

In [2]:
apn.__version__

'1.3'

First, we have to read again the data used to perform the fit. We also need to read the a2z DataFrame that `apollinaire` created at the end of the MCMC process.

In [3]:
t, v = apn.timeseries.load_light_curve (star='006603624')

dt = np.median (t[1:] - t[:-1]) * 86400
freq, psd = apn.psd.series_to_psd (v, dt=dt, correct_dc=True)
freq, psd = freq*1e6, psd*1e-6
back = np.loadtxt ('first_steps_tutorial/background.dat')
df_a2z = apn_pkb.read_a2z ('first_steps_tutorial/modes_param.a2z')

The Bayes factor is computed considering pairs of modes, even $\ell = \{0,2\}$ or odd $\ell = \{1,3\}$. Let's extract from the DataFrame the mode orders on which we performed the fit.

In [4]:
orders = apn_pkb.get_order_list (df_a2z)

We now have to loop over the orders to compute the Bayes factor for each pair of modes.

In [5]:
bf_array = np.zeros ((orders.size*4, 3))

for ii, n in enumerate (orders) :
    psw, ps, p0, _, = apn_pkb.bayes_factor (freq, psd, back, df_a2z, 
                                            n, strategy='order', 
                                            size_window=45,
                                            l02=True, thin=500, 
                                            discard=500, instr='geometric', 
                                            hdf5Dir='first_steps_tutorial', 
                                            add_ampl=True, 
                                            parallelise=True, nodes=8)

    lnKsw, lnKs = apn_pkb.compute_lnK (psw, ps, p0) 
    bf_array[ii*4,:] = n, 0, lnKs
    bf_array[ii*4+1,:] = n-1, 2, lnKsw

    psw, ps, p0, _, = apn_pkb.bayes_factor (freq, psd, back, df_a2z, 
                                            n, strategy='order', 
                                            size_window=45,
                                            l02=False, thin=500, 
                                            discard=500, instr='geometric', 
                                            hdf5Dir='first_steps_tutorial', 
                                            add_ampl=True, 
                                            parallelise=True, nodes=8)

    lnKsw, lnKs = apn_pkb.compute_lnK (psw, ps, p0)
    bf_array[ii*4+2,:] = n, 1, lnKs
    bf_array[ii*4+3,:] = n-1, 3, lnKsw

Window width: 45.0 muHz, with low bound at 2121.4 muHz and up bound at 2166.4 muHz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:06<00:00, 237.89it/s]


Window width: 45.0 muHz, with low bound at 2170.7 muHz and up bound at 2215.7 muHz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:02<00:00, 726.77it/s]


Window width: 45.0 muHz, with low bound at 2231.8 muHz and up bound at 2276.8 muHz


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:02<00:00, 683.75it/s]


Window width: 45.0 muHz, with low bound at 2562.8 muHz and up bound at 2607.8 muHz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:02<00:00, 677.76it/s]


Window width: 45.0 muHz, with low bound at 2613.3 muHz and up bound at 2658.3 muHz


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:02<00:00, 659.57it/s]


Let's display the results. Having $\ln K = \infty$ means that all tested models were favoured against H0 (and $\ln K = - \infty$ means that H0 was favoured against every tested model). 

In [6]:
quality = pd.DataFrame (data=bf_array[:,2],
                        index=pd.MultiIndex.from_arrays (np.transpose (bf_array[:,:2].astype (np.int_))),
                        columns=['ln K'])

quality

,,ln K
18,0,inf
17,2,inf
18,1,inf
17,3,-inf
19,0,inf
18,2,inf
19,1,inf
18,3,inf
20,0,inf
19,2,inf
